<a href="https://colab.research.google.com/github/ElenaPribylova/-10/blob/main/%D0%90%D0%BA%D1%82%D0%B8%D0%B2%D0%BD%D0%BE%D1%81%D1%82%D1%8C_Itresume.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **2 кейс**

**Выгрузка активности с ItResume**

**Важно**

Перед началом решения выполните следующую ячейку, чтобы загрузить необходимый для работы файл.

In [4]:
!wget https://gist.github.com/Vs8th/a7a7f00e6cdef1b3fe87e4d61ca56e5f/raw/codesubmit.csv

--2025-12-06 15:07:08--  https://gist.github.com/Vs8th/a7a7f00e6cdef1b3fe87e4d61ca56e5f/raw/codesubmit.csv
Resolving gist.github.com (gist.github.com)... 140.82.112.4
Connecting to gist.github.com (gist.github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://gist.githubusercontent.com/Vs8th/a7a7f00e6cdef1b3fe87e4d61ca56e5f/raw/codesubmit.csv [following]
--2025-12-06 15:07:09--  https://gist.githubusercontent.com/Vs8th/a7a7f00e6cdef1b3fe87e4d61ca56e5f/raw/codesubmit.csv
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 215378 (210K) [text/plain]
Saving to: ‘codesubmit.csv’

codesubmit.csv      100%[===================>] 210.33K  --.-KB/s    in 0.03s   

2025-12-06 15:07:09 (7.95 MB/s) - ‘co

Чтобы посмотреть как он выглядит выполните следующую ячейку.

In [5]:
import pandas as pd

df = pd.read_csv('codesubmit.csv', sep = ';')
df

,created_at,user_id,problem_id,is_correct,type
0,2023-04-30 13:47:14.344471,7,870,1.0,submit
1,2023-04-30 13:46:15.949925,7,870,0.0,submit
2,2023-04-30 16:13:26.005286,173,21,1.0,submit
3,2023-04-30 16:13:06.739782,173,21,NaN,run
4,2023-04-30 15:52:00.195532,173,25,1.0,submit
...,...,...,...,...,...
4994,2023-04-30 21:52:00.269123,13493,435,NaN,run
4995,2023-04-30 21:51:01.094234,13493,435,1.0,submit
4996,2023-04-30 21:50:52.059690,13493,435,NaN,run
4997,2023-04-30 21:42:24.323689,13493,1086,NaN,run


### **Решения**

#### **Задача 1**

Ваша задача - выяснить сколько в среднем тратится времени на решение задачи.

**Примечание**: для правильного подсчета - рассчитайте сначала среднее время решения по каждой задаче в отдельности, и только затем находите общее среднее время решения задач.

Результат - число типа `float`, округлите до 2 знаков после запятой и запишите в переменную `res`.


**Решение**

Напишите свое решение ниже

In [6]:
import csv
from collections import defaultdict
from datetime import datetime

filename = "codesubmit.csv"

# Шаг 1: Чтение данных
rows = []
with open(filename, 'r', encoding='utf-8') as csvfile:
    csvreader = csv.DictReader(csvfile, delimiter=';')
    for row in csvreader:
        if row:  # Пропускаем пустые строки
            rows.append(row)

# Шаг 2: Группировка и фильтрация решённых задач
grouped_data = defaultdict(list)
for row in rows:
    user_id = row['user_id']
    problem_id = row['problem_id']
    key = (user_id, problem_id)
    grouped_data[key].append({
        'created_at': row['created_at'],
        'is_correct': row['is_correct']
    })

# Шаг 3: Расчёт времени для решённых задач
result = {}  # {user_id: {problem_id: time_in_seconds}}
zero_time_problems = []  # Задачи, решённые сразу

for (user_id, problem_id), submissions in grouped_data.items():
    # Сортируем по времени
    sorted_subs = sorted(submissions, key=lambda x: x['created_at'])

    # Находим первую верную попытку
    first_correct = None
    for sub in sorted_subs:
        if sub['is_correct'] == '1':
            first_correct = sub
            break

    if not first_correct:
        continue  # Задача не решена

    # Время первой попытки
    first_time = datetime.strptime(sorted_subs[0]['created_at'], '%Y-%m-%d %H:%M:%S.%f')
    # Время первой верной попытки
    correct_time = datetime.strptime(first_correct['created_at'], '%Y-%m-%d %H:%M:%S.%f')

    time_spent = (correct_time - first_time).total_seconds()

    if time_spent == 0:
        zero_time_problems.append((user_id, problem_id))
    else:
        if user_id not in result:
            result[user_id] = {}
        result[user_id][problem_id] = time_spent

# Шаг 4: Среднее время по задачам
time_spent_by_problem = defaultdict(lambda: {'total': 0, 'count': 0})

for user_id, problems in result.items():
    for problem_id, time in problems.items():
        time_spent_by_problem[problem_id]['total'] += time
        time_spent_by_problem[problem_id]['count'] += 1

# Среднее время на задачу
average_time_spent = {
    problem: data['total'] / data['count']
    for problem, data in time_spent_by_problem.items()
}

# Общее среднее время
if average_time_spent:
    avg_value = sum(average_time_spent.values()) / len(average_time_spent)
    res = round(avg_value, 2)
else:
    res = 0.0  # Если нет решённых задач

print("Среднее время решения задачи:", res)
print("Задачи, решённые сразу:", zero_time_problems)


Среднее время решения задачи: 611.86
Задачи, решённые сразу: [('546', '12'), ('1259', '15'), ('1339', '756'), ('1388', '152'), ('1388', '153'), ('1388', '154'), ('1388', '156'), ('1388', '160'), ('1388', '180'), ('1388', '181'), ('1388', '182'), ('1973', '877'), ('2197', '1087'), ('2197', '1089'), ('2197', '1090'), ('2435', '32'), ('2435', '121'), ('2435', '137'), ('2535', '1005'), ('2535', '1006'), ('2621', '1151'), ('2621', '1152'), ('2684', '1164'), ('2687', '1144'), ('2714', '884'), ('3089', '172'), ('3984', '156'), ('4464', '49'), ('4464', '50'), ('4464', '52'), ('4464', '53'), ('4464', '87'), ('4464', '88'), ('4464', '89'), ('4496', '93'), ('4633', '49'), ('4633', '50'), ('4633', '52'), ('5072', '1164'), ('5720', '16'), ('6497', '16'), ('6624', '837'), ('6657', '154'), ('6657', '160'), ('6657', '161'), ('6657', '164'), ('6657', '165'), ('6733', '148'), ('7065', '842'), ('7276', '17'), ('7276', '33'), ('7276', '36'), ('7276', '131'), ('7276', '132'), ('7276', '133'), ('7276', '135

✏️ ✏️ ✏️

**Проверка**

Чтобы проверить свое решение, запустите код в следующих ячейках

In [7]:
try:
    assert res == 611.86
except AssertionError:
    print('Ответы не совпадают')
else:
    print('Поздравляем, Вы справились!')

Поздравляем, Вы справились!


#### **Задача 2**

Ваша задача - выяснить сколько часов в среднем проводит юзер в день на платформе. Перерывы в активности за день - не учитываем.

Результат - число типа `float`, округлите до 2 знаков после запятой и запишите в переменную `res2`.

**Решение**

Напишите свое решение ниже

In [8]:
import statistics
import datetime

user_attempts = {}
for attempt in rows:
    user_id = attempt['user_id']
    if user_id not in user_attempts:
        user_attempts[user_id] = []
    user_attempts[user_id].append(attempt)

# Сортируем попытки каждого пользователя по времени
for user_id in user_attempts:
    user_attempts[user_id].sort(key=lambda x: datetime.datetime.strptime(
        x['created_at'], '%Y-%m-%d %H:%M:%S.%f'
    ))

user_time = {}
for user_id, attempts in user_attempts.items():
    if len(attempts) == 0:
        user_time[user_id] = 0
        continue

    # Берём первую и последнюю попытку
    first_attempt = datetime.datetime.strptime(
        attempts[0]['created_at'], '%Y-%m-%d %H:%M:%S.%f'
    )
    last_attempt = datetime.datetime.strptime(
        attempts[-1]['created_at'], '%Y-%m-%d %H:%M:%S.%f'
    )

    # Общее время в часах: разница между первой и последней попыткой
    total_seconds = (last_attempt - first_attempt).total_seconds()
    user_time[user_id] = total_seconds / 3600  # в часах

# Проверяем, что есть данные для расчёта среднего
if user_time:
    average_time = statistics.mean(user_time.values())
    res2 = round(average_time, 2)
else:
    res2 = 0  # или другое значение по умолчанию


✏️ ✏️ ✏️

**Проверка**

Чтобы проверить свое решение, запустите код в следующих ячейках

In [9]:
try:
    assert res2 == 1.7
except AssertionError:
    print('Ответы не совпадают')
else:
    print('Поздравляем, Вы справились!')

Поздравляем, Вы справились!


#### **Задача 3**

Теперь давайте посмотрим на активные сеансы. Выясните, сколько задач в среднем решается за один активный сеанс.

**Активный сеанс** - период, когда между любой активностью пользователя разница менее или равна часу, не более

**Важно**: в расчет берем не только успешные попытки решений (`is_correct=1`), а и неуспешные тоже (`is_correct=0`), и тип `run` в том числе.

Результат - число типа `float`, округлите до 2 знаков после запятой и запишите в переменную `res3`.

**Решение**

Напишите свое решение ниже

In [ ]:

# Ваше решение здесь

...


res3 =

In [10]:
import datetime
from datetime import timedelta

# Сортируем строки по пользователю и времени
rows.sort(key=lambda x: (x['user_id'], x['created_at']))

dates_by_user = {}
for row in rows:
    user_id = row['user_id']
    problem_id = row['problem_id']
    created_at = datetime.datetime.strptime(row['created_at'], '%Y-%m-%d %H:%M:%S.%f')
    if user_id not in dates_by_user:
        dates_by_user[user_id] = []
    dates_by_user[user_id].append((created_at, problem_id))

results = {}

for user, submissions in dates_by_user.items():
    if not submissions:  # Обработка пустого списка
        continue

    sessions = []
    current_session = [submissions[0]]  # Первая попытка всегда начинает сессию

    # Формируем сессии (разделение при интервале > 1 часа)
    for i in range(1, len(submissions)):
        time_diff = submissions[i][0] - submissions[i-1][0]
        if time_diff > timedelta(hours=1):
            sessions.append(current_session)
            current_session = [submissions[i]]
        else:
            current_session.append(submissions[i])

    # Добавляем последнюю сессию
    sessions.append(current_session)

    # Инициализируем результат для пользователя
    results[user] = {
        'sessions': len(sessions),
        'problems_solved_per_session': []
    }

    # Считаем уникальные задачи в каждой сессии
    for session in sessions:
        unique_problems = set()  # Используем set для уникальности
        for submission in session:
            unique_problems.add(submission[1])  # problem_id
        results[user]['problems_solved_per_session'].append(len(unique_problems))

# Агрегируем результаты
total_sessions = 0
total_problems_solved = 0

for user in results:
    total_sessions += results[user]['sessions']
    total_problems_solved += sum(results[user]['problems_solved_per_session'])

# Рассчитываем среднее
if total_sessions > 0:
    average_problems_solved_per_session = total_problems_solved / total_sessions
    res3 = round(average_problems_solved_per_session, 2)
else:
    res3 = 0.0  # Если сессий нет


✏️ ✏️ ✏️

**Проверка**

Чтобы проверить свое решение, запустите код в следующих ячейках

In [11]:
try:
    assert res3 == 3.14
except AssertionError:
    print('Ответы не совпадают')
else:
    print('Поздравляем, Вы справились!')

Поздравляем, Вы справились!


#### **Задача 4**

И финальная - найдите самый "популярный" час дня на нашей платформе.

Популярность определяем максимальным количеством уникальных пользователей, совершающих какую-либо активность в этот период

Результат в числовом формате запишите в переменную `res4`.

Например, самым популярным часом стал период с 22 до 23, тогда в переменной `res4` должно лежать **22**. Обозначающее начало этого периода.

**Решение**

Напишите свое решение ниже

In [12]:
from collections import defaultdict
from datetime import datetime

# Группируем user_id по часам
hour_users = defaultdict(set)
for d in rows:
    hour = datetime.strptime(d['created_at'], '%Y-%m-%d %H:%M:%S.%f').hour
    hour_users[hour].add(d['user_id'])

# Считаем активность по часам
activity_by_hour = {hour: len(users) for hour, users in hour_users.items()}

# Находим час с максимумом
max_count = max(activity_by_hour.values())
max_activity_hours = [hour for hour, count in activity_by_hour.items() if count == max_count]
res4 = max_activity_hours[0]


✏️ ✏️ ✏️

**Проверка**

Чтобы проверить свое решение, запустите код в следующих ячейках

In [13]:
try:
    assert res4 == 16
except AssertionError:
    print('Ответы не совпадают')
else:
    print('Поздравляем, Вы справились!')

Поздравляем, Вы справились!
